## CS484 Final Project: Image inpainting using Object removal

Hello all
This is the showcase of our project

#### Section 1: Data gathering and cleaning
downloading the data and storing it on local file made easy. might create a seprate python file to streamline the process on the notebook ? 

In [2]:
! pip install deeplake

In [ ]:
from torch.utils.data import DataLoader
import torchvision.io as tvio
import torchvision.transforms as tfs
import torchvision.transforms.functional as tF

from download_dataset import DatasetDownloader
import numpy as np
import matplotlib.pyplot as plt

from utility_functions import RectanglePatch, RandomHFlip, RandomVFlip, PairColorJitter, RandomResizeCrop, PairToTensors, PairNormalize, MakePair
from torch.utils.data import Dataset

c:\Users\qhuyn\anaconda3\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.0.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
-

Opening dataset in read-only mode as you don't have write permissions.


This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/places205



/

hub://activeloop/places205 loaded successfully.



HINT: Please forward the port - 11685 to your local machine, if you are running on the cloud.


 * Serving Flask app 'dataset_visualizer'
 * Debug mode: off


In [ ]:
#ds = DatasetDownloader.get_places()
#ds.visualize()

#### Section 2: Object detection and removal
for this section we will be using either two methods for patch selection:
- simple rectangle patch
- Object selection using deep segmentaiton methods with the help  of labels like in A5 ? (optional) 

The goal of this section is to create the dataset that is ready to be used for training/validating. for each image in the data set we should have an (Image - patch, Image) in the final dataset, ready to be trained (or in the least, the region to be removed is somehow indicated?)

In [ ]:

training_img_transform = tfs.Compose([  
  MakePair(),
  RandomHFlip(),
  RandomVFlip(),
  PairColorJitter(),
  RandomResizeCrop(size=256, min_scale=0.7, max_scale=1.3),
  RectanglePatch(patch_size=(100,100)),
  # PairToTensors()
  # PairNormalize(mean = ???, std = ???)
  ])

validation_img_transform = tfs.Compose([
    MakePair(),
    PairToTensors(),
   # PairNormalize( mean = ???, std = ???)
])

In [ ]:
ds = DatasetDownloader.get_places()
training_data, validation_data = ds.random_split([0.8, 0.2])

class CustomPlaces(Dataset):
    def __init__(self, data, transform=None):

        self.ds = data
        self.transform = transform
        train_dataset, test_dataset = torch.utils.data.random_split(self.ds, [0.8, 0.2])

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        img = ds.images[idx].numpy().copy()
        withPatch, target = self.transform(img)
        return withPatch, target

training_ds = CustomPlaces(data = training_data, transform=training_img_transform)
validating_ds = CustomPlaces(data = validation_data, transform=validation_img_transform)

In [ ]:
#feel free to delete
x = training_ds[3]

plt.figure(figsize=(20, 8))
plt.subplot(131)
plt.imshow(x[0], vmin=0, vmax=255)
plt.title('with patch')
plt.axis('off')

plt.subplot(132)
plt.imshow(x[1], vmin=0, vmax=255)
plt.title('target')
plt.axis('off')

In [ ]:
# these are just copied from A5
TRAIN_BATCH_SIZE = 4
NUM_WORKERS = 0

train_loader = DataLoader(training_ds, batch_size=TRAIN_BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
val_loader = DataLoader(validating_ds, batch_size=1, num_workers=NUM_WORKERS, shuffle=False)

#### Section 3: Creating the  model for the network
the first simple approach would be to use an autoencoder and self-supervision to fit the model to the dataset.

questions:
- how good would the image quality be?
- what if we change the location of the patch?
- how would loss functions affect the quality of the output?
- what if patch was not rectangular?
- how to avoid overfitting?

#### Section 3.1: Training the model, displaying the results

#### Section 4: GANS, creating competition for the network using a "detector"

#### Section 4.1: Training the model, displaying the results

#### Section 5(if we had time): using image segmentation from assignment 4 part 2, to remove objects from image and using network from before to fill in the "patch"